Following Issues resolved:

1) test_image_14 : two squares treated as one, due to which undefined centroid key value in color dict, also the only **orange square** being detected is giving an answer- **Red** *(refer issue and issue1 images)*

2) test_image_15 : **one orange triangle** detected as **blue** triangle *(refer issue2 image)*

In [23]:
import cv2
import numpy as np
import os
##############################################################

################# ADD UTILITY FUNCTIONS HERE #################

def centroid_cnt(contour):
    """
    Purpose:
    ---
    This function takes contour as an argument and returns a list containing
    the x and y values of the centroid

    Input Arguments:
    ---
    `contour` : [ numpy array ]
            numpy array of (x, y) coordinates of boundary points of the object.

    Returns:
    ---
    `centroid` : [ list ]
            list of (x,y) coordinates of the centroid of the contours
    """
    
    centroid = []

    M = cv2.moments(contour)
    if M['m00'] != 0.0:
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])

        centroid.append((cx,cy))

    return centroid

def centroid_img(img):
    """
    Purpose:
    ---
    This function takes image as an argument and returns a list containing
    the x and y values of the centroid

    Input Arguments:
    ---
    `contour` : [ numpy array ]
            numpy array of (x, y) coordinates of boundary points of the object.

    Returns:
    ---
    `centroid` : [ list ]
            list of (x,y) coordinates of the centroid of the contours
    """
    centroid = []

    # converting image to grayscale
    image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Thresholding
    _, threshold = cv2.threshold(image, 200, 255, cv2.THRESH_BINARY)

    # Contouring
    contours,_ = cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # to loop over the contours
    i=0
    for cnt in contours:
    
        # here we are ignoring first counter because 
        # findContours function detects whole image as shape
        if i==0:
            i=1
            continue

        M = cv2.moments(cnt)
        if M['m00'] != 0.0:
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])

        centroid.append((cx,cy))

    return centroid


def colors_detected(img):
    """
    Purpose:
    ---
    This function takes the image as argument and returns a dictionary
    denoting the color of the shapes in the image.

    Input Arguments:
    ---
    `img` : [ numpy array ]
            numpy array of image returned by cv2 library

    Returns:
    ---
    `detected_colors` : {dictionary}
            dictionary containing details of colors present in image
    """
    detected_colors = {}

    centroids = centroid_img(img)

    for cent in centroids:
        
        # centroid    
        cx = cent[0]
        cy = cent[1]

        # finding out blue, green and red values for the pixel
    
        # image[y, x] is the correct syntax based on the fact that 
        # the x-value is the column number (i.e., width), and 
        # the y-value is the row number (i.e., height).
        (b, g, r) = img[cy][cx]
        
        if (b==0 and g==0 and r==255):
            detected_colors[cx,cy] = ['Red']
        elif (b==0 and g==255 and r==0):
            detected_colors[cx,cy] = ['Green']
        elif (b==255 and g==0 and r==0):
            detected_colors[cx,cy] = ['Blue']
        else:
            detected_colors[cx,cy] = ['Orange']

    return detected_colors


def shapes(img):
    
    img_shapes = {}
    # converting image to grayscale
    image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Thresholding
    _, threshold = cv2.threshold(image, 200, 255, cv2.THRESH_BINARY)

    # Contouring
    contours,_ = cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    #to loop over the contours
    i=0
    for cnt in contours:
    
        # ignoring first counter because 
        # findcontour function detects whole image as one shape
        if i==0:
            i=1
            continue

        # approximating
        approx = cv2.approxPolyDP(cnt, 0.025*cv2.arcLength(cnt, True), True)
        

        centroids = centroid_cnt(cnt)

        # centroid    
        cx = centroids[0][0]
        cy = centroids[0][1]
        
        # putting shape name
        if len(approx) == 3:
            img_shapes[cx,cy] = ['Triangle']
  
        elif len(approx) == 4:
        
            # for distinguishing between rectangle and square
            x, y, w, h = cv2.boundingRect(approx)

            aspectRatio = float (w)/h
            if aspectRatio >= 0.95 and aspectRatio <= 1.05:
                img_shapes[cx,cy] = ['Square']
            else:
                img_shapes[cx,cy] = ['Rectangle']

        elif (len(approx) == 5):
             img_shapes[cx,cy] = ['Pentagon']
  
        else:
             img_shapes[cx,cy] = ['Circle']

    return img_shapes


##############################################################

def detect_shapes(img):

	"""
	Purpose:
	---
	This function takes the image as an argument and returns a nested list
	containing details of colored (non-white) shapes in that image

	Input Arguments:
	---
	`img` :	[ numpy array ]
			numpy array of image returned by cv2 library

	Returns:
	---
	`detected_shapes` : [ list ]
			nested list containing details of colored (non-white) 
			shapes present in image
	
	Example call:
	---
	shapes = detect_shapes(img)
	"""    
	detected_shapes = []

	##############	ADD YOUR CODE HERE	##############
    
	color = colors_detected(img)

	shape = shapes(img)

# 	print(color)
    
# 	print(shape)
    
	keys_color = list(color.keys())
    
	keys_shape = list(shape.keys())
    
# 	print(keys_color)
    
# 	print(keys_shape)
    
	details_list = []

	for key_shp in keys_shape:
# 		print(key_shp[0])
# 		print(key_shp[1])
		details_list = []
		cx_key = key_shp[0]
		cy_key = key_shp[1]
        
#         # error inclusion
# 		upper_cx = 1.01 * cx_key  
# #         upper_cx = cx_key + 0.01 * cx_key
# 		lower_cx = 0.99 * cx_key
    
# 		upper_cy = 1.01 * cy_key  
# 		lower_cy = 0.99 * cy_key

		for key_clr in keys_color:
			cx_k = key_clr[0]
			cy_k = key_clr[1]
            

# 			if (lower_cx <= cx_k <= upper_cx or lower_cy <= cy_k <= upper_cy):
			if (cx_k == cx_key and cy_k == cy_key):
				temp1 = (cx_k, cy_k)
				temp2 = (cx_key, cy_key)
# 				print(temp1)
# 				print(temp2)
				details_list.append(color.get(temp1)[0])
				details_list.append(shape.get(temp2)[0])
				details_list.append(temp2)
				detected_shapes.append(details_list)
				if len(details_list) == 3 :
					break
# 				print(detected_shapes)
    
	for detected in detected_shapes:
		colour = detected[0]
		shape = detected[1]
		coordinates = detected[2]
		cv2.putText(img, str((colour, shape)),coordinates, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2)
		cv2.imshow("labeled_image", img)
		cv2.waitKey(0)
		cv2.destroyAllWindows()
	##################################################

    
    
	return detected_shapes

In [24]:
def get_labeled_image(img, detected_shapes):
	######### YOU ARE NOT ALLOWED TO MAKE CHANGES TO THIS FUNCTION #########
	"""
	Purpose:
	---
	This function takes the image and the detected shapes list as an argument
	and returns a labelled image

	Input Arguments:
	---
	`img` :	[ numpy array ]
			numpy array of image returned by cv2 library

	`detected_shapes` : [ list ]
			nested list containing details of colored (non-white) 
			shapes present in image

	Returns:
	---
	`img` :	[ numpy array ]
			labelled image
	
	Example call:
	---
	img = get_labeled_image(img, detected_shapes)
	"""
	######### YOU ARE NOT ALLOWED TO MAKE CHANGES TO THIS FUNCTION #########    

	for detected in detected_shapes:
		colour = detected[0]
		shape = detected[1]
		coordinates = detected[2]
		cv2.putText(img, str((colour, shape)),coordinates, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2)
	return img

In [25]:
img = cv2.imread('test_images/test_image_1.png')
print(detect_shapes(img))

[['Red', 'Circle', (588, 370)], ['Green', 'Square', (325, 145)]]


In [26]:
img = cv2.imread('test_images/test_image_2.png')
print(detect_shapes(img))

[['Blue', 'Circle', (274, 348)], ['Blue', 'Pentagon', (648, 420)], ['Green', 'Triangle', (766, 165)], ['Red', 'Rectangle', (301, 118)]]


In [27]:
img = cv2.imread('test_images/test_image_3.png')
print(detect_shapes(img))

[['Green', 'Rectangle', (467, 467)], ['Blue', 'Square', (425, 272)], ['Green', 'Circle', (290, 110)], ['Red', 'Triangle', (643, 194)]]


In [28]:
img = cv2.imread('test_images/test_image_4.png')
print(detect_shapes(img))

[['Blue', 'Square', (325, 451)], ['Red', 'Rectangle', (249, 199)], ['Blue', 'Circle', (588, 226)]]


In [29]:
img = cv2.imread('test_images/test_image_5.png')
print(detect_shapes(img))

[['Red', 'Rectangle', (648, 398)], ['Green', 'Circle', (308, 370)], ['Blue', 'Circle', (631, 149)], ['Blue', 'Square', (183, 105)]]


In [30]:
img = cv2.imread('test_images/test_image_6.png')
print(detect_shapes(img))

[['Blue', 'Circle', (274, 348)], ['Blue', 'Pentagon', (648, 420)], ['Green', 'Triangle', (766, 165)], ['Red', 'Rectangle', (301, 118)]]


In [31]:
img = cv2.imread('test_images/test_image_7.png')
print(detect_shapes(img))

[['Blue', 'Square', (183, 491)], ['Blue', 'Pentagon', (545, 457)], ['Red', 'Rectangle', (648, 199)], ['Red', 'Circle', (308, 226)], ['Green', 'Pentagon', (103, 108)]]


In [32]:
img = cv2.imread('test_images/test_image_8.png')
print(detect_shapes(img))

[['Blue', 'Circle', (295, 503)], ['Red', 'Pentagon', (517, 352)], ['Red', 'Pentagon', (160, 234)], ['Green', 'Triangle', (660, 117)], ['Blue', 'Square', (101, 86)]]


In [33]:
img = cv2.imread('test_images/test_image_9.png')
print(detect_shapes(img))

[['Blue', 'Circle', (295, 503)], ['Blue', 'Pentagon', (517, 352)], ['Green', 'Rectangle', (197, 246)], ['Green', 'Triangle', (660, 117)], ['Blue', 'Square', (101, 86)]]


In [34]:
img = cv2.imread('test_images/test_image_10.png')
print(detect_shapes(img))

[['Red', 'Circle', (788, 476)], ['Red', 'Rectangle', (420, 453)], ['Red', 'Circle', (107, 340)], ['Blue', 'Triangle', (184, 212)], ['Blue', 'Triangle', (91, 89)]]


In [35]:
img = cv2.imread('test_images/test_image_11.png')
print(detect_shapes(img))

[['Blue', 'Rectangle', (467, 467)], ['Blue', 'Square', (425, 272)], ['Red', 'Circle', (290, 110)], ['Red', 'Triangle', (643, 194)]]


In [36]:
img = cv2.imread('test_images/test_image_12.png')
print(detect_shapes(img))

[['Red', 'Circle', (788, 476)], ['Red', 'Rectangle', (420, 453)], ['Red', 'Circle', (107, 340)], ['Blue', 'Triangle', (184, 212)], ['Blue', 'Triangle', (91, 89)]]


In [40]:
img = cv2.imread('test_images/test_image_13.png')
print(detect_shapes(img))

[['Orange', 'Triangle', (452, 504)], ['Green', 'Square', (778, 482)], ['Blue', 'Rectangle', (650, 401)], ['Red', 'Circle', (529, 325)], ['Red', 'Rectangle', (121, 220)], ['Blue', 'Triangle', (646, 199)]]


In [41]:
img = cv2.imread('test_images/test_image_14.png')
print(detect_shapes(img))

[['Blue', 'Triangle', (254, 340)], ['Orange', 'Square', (308, 248)], ['Red', 'Circle', (215, 247)], ['Orange', 'Square', (372, 182)], ['Red', 'Circle', (161, 183)], ['Red', 'Circle', (101, 101)], ['Orange', 'Square', (455, 98)]]


In [42]:
img = cv2.imread('test_images/test_image_15.png')
print(detect_shapes(img))

[['Blue', 'Circle', (82, 509)], ['Orange', 'Triangle', (601, 506)], ['Blue', 'Circle', (317, 320)], ['Orange', 'Triangle', (602, 234)], ['Red', 'Square', (201, 218)], ['Blue', 'Circle', (292, 161)], ['Red', 'Square', (395, 165)], ['Red', 'Square', (203, 98)], ['Blue', 'Circle', (793, 96)]]
